In [1]:
import os
os.environ.setdefault("OMP_NUM_THREADS", "1")
os.environ.setdefault("OPENBLAS_NUM_THREADS", "1")
os.environ.setdefault("MKL_NUM_THREADS", "1")
# os.environ.setdefault("NUMEXPR_NUM_THREADS", "1")
import re
from pathlib import Path
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
# --- Ajustes recomendados para Matplotlib en modo no interactivo ---
matplotlib.use("Agg")  # evita backends GUI en nodos de cómputo
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from concurrent.futures import ProcessPoolExecutor, as_completed
import multiprocessing as mp
import gc

# ============================================================
# Comparador de resultados UAMEX vs UNISONs
# ============================================================

# =============== CONFIGURACIÓN DE RUTAS ===============
RUTA_BASE          = Path("/lustre/home/mvalenzuela/Ocotillo/DataAqua")

RUTA_SALIDA_UNISON = RUTA_BASE / "Salidas_ETo12" / "Periodo de Cultivo ETo"
RUTA_SALIDA_UAMEX  = RUTA_BASE / "Datos" / "ET Calculada - Por Periodo de Cultivo - 2010-2024"
RUTA_COMP          = RUTA_BASE / "Comparaciones_FAO56_7"
RUTA_COMP.mkdir(parents=True, exist_ok=True)

# Limpieza: borra PNGs previos de comparaciones/heatmaps (no generaremos PNGs nuevos)
for p in list(RUTA_COMP.glob("*_comparacion.png")) + list(RUTA_COMP.glob("heatmap_*.png")):
    try:
        p.unlink()
    except Exception:
        pass

# =============== VARIABLES A COMPARAR ===============
# (quitamos Rso porque solo existe en UNISON)
VARS = ["Rnl", "ET0"]  # <-- SOLO estas se grafican/comparan y se reportan

# =============== NORMALIZACIÓN DE REGIONES ===============
REGION_FIX_UAMEX = {
    "CAJEME": "Cajeme",
    "ETCHOJOA": "Etchojoa",
    "ENSENADA": "Ensenada",
    "ZAPOPAN": "Zapopan",
    "TOLUCA": "Toluca",
    "METEPEC": "Metepec",
    "VILLADEALLENDE": "Villa de Allende",
    "VILLA_DE_ALLENDE": "Villa de Allende",
}
REGION_FIX_UNISON = {
    "VillaAllende": "Villa de Allende",
    "Etchhojoa": "Etchojoa",
}

# =============== MAPEO DE COLUMNAS A NOMBRES CANÓNICOS ===============
# UAMEX (ellos, Excel)
MAP_UAMEX = {
    "Día": "Dia", "Dia": "Dia",
    "Tmax": "Tmax", "Tmin": "Tmin",
    "HR": "HR", "Ux": "Ux",
    "Rs": "Rs", "Pef": "Pef",
    "Tmean": "Tmean", "es": "es", "ea": "ea",
    "delta": "delta", "P": "P", "gamma": "gamma",
    "Rns": "Rns", "Rnl": "Rnl", "Rn": "Rn",
    "ET0": "ET0", "Kc": "Kc", "ETc": "ETc", "decada": "decada",
    "ETverde": "ETverde", "ETazul": "ETazul",
    # posibles extras
    "Rl": "Rl", "Rso": "Rso", "Ptot": "Ptot",
}

# UNISON (nosotros, CSV)
MAP_UNISON = {
    # Mojibake/variantes
    "Año_ (YEAR)": "Year", "AÃ±o_ (YEAR)": "Year",
    "Día (DOY)": "Dia", "DÃ­a (DOY)": "Dia",

    # Base NASA
    "Tmax (T2M_MAX)": "Tmax",
    "Tmin (T2M_MIN)": "Tmin",
    "HR (RH2M)": "HR",
    "Ux (WS2M)": "Ux",
    "Rs (ALLSKY_SFC_SW_DWN)": "Rs",
    "Rl_ (ALLSKY_SFC_LW_DWN)": "Rl",
    "Ptot_ (PRECTOTCORR)": "Ptot",

    # Generadas / resultados (quitar sufijo _)
    "Pef_": "Pef", "Tmean_": "Tmean", "es_": "es", "ea_": "ea",
    "delta_": "delta", "P_": "P", "gamma_": "gamma",
    "Rns_": "Rns", "Rnl_": "Rnl", "Rn_": "Rn", "Rso_": "Rso",
    "Kc_": "Kc", "decada_": "decada",

    # Resultados
    "ET0": "ET0", "ETc": "ETc", "ETverde": "ETverde", "ETazul": "ETazul",

    # por si ya venían limpios
    "Year": "Year", "Dia": "Dia",
}

# =============== PARSERS DE NOMBRE DE ARCHIVO ===============
def parse_uamex_filename(path: str):
    """
    UAMEX: '2011-2012-ET_CALCULADA_CAJEME.xlsx' -> ('Cajeme', '2011-2012')
           '2013-ET_CALCULADA_ZAPOPAN.xlsx'     -> ('Zapopan','2013')
    """
    f = os.path.basename(path)
    m = re.match(r"(\d{4})(?:[-_](\d{4}))?[-_]?ET.*CALCULADA[-_]?([A-Za-z_]+)\.xlsx$", f, re.I)
    if not m:
        return None, None
    y1, y2, reg = m.groups()
    reg_norm = REGION_FIX_UAMEX.get(reg.upper(), reg.title())
    years = y1 if not y2 else f"{y1}-{y2}"
    return reg_norm, years

def parse_unison_filename(path: str):
    """
    UNISON: 'Cajeme-FAO56-2011-2012-SALIDA.csv' -> ('Cajeme', '2011-2012')
            'Zapopan-FAO56-2014-SALIDA.csv'     -> ('Zapopan','2014')
    """
    f = os.path.basename(path)
    m = re.match(r"([A-Za-z]+)-FAO56-(\d{4})(?:-(\d{4}))?-SALIDA\.csv$", f, re.I)
    if not m:
        return None, None
    reg, y1, y2 = m.groups()
    reg_norm = REGION_FIX_UNISON.get(reg, reg)
    years = y1 if not y2 else f"{y1}-{y2}"
    return reg_norm, years

# =============== LECTORES ===============
def robust_read_excel_uamex(path: Path) -> pd.DataFrame:
    """Lee Excel UAMEX, normaliza encabezados y fuerza Dia=1..N."""
    df = pd.read_excel(path, engine="openpyxl")
    df.columns = [c.strip() for c in df.columns]
    df = df.rename(columns=lambda c: MAP_UAMEX.get(c, c))
    df["Dia"] = np.arange(1, len(df) + 1)
    for c in set(VARS + ["Dia"]).intersection(df.columns):
        df[c] = pd.to_numeric(df[c], errors="coerce")
    return df

def robust_read_csv_unison(path: Path) -> pd.DataFrame:
    """Lee CSV UNISON, corrige mojibake/sufijos, fuerza Dia=1..N."""
    last_err = None
    for enc in ("utf-8", "latin-1"):
        try:
            df = pd.read_csv(path, encoding=enc)
            last_err = None
            break
        except UnicodeDecodeError as e:
            last_err = e
            continue
    if last_err is not None:
        df = pd.read_csv(path)
    df.columns = [c.strip() for c in df.columns]
    df = df.rename(columns=lambda c: MAP_UNISON.get(c, c))
    df["Dia"] = np.arange(1, len(df) + 1)
    for c in set(VARS + ["Dia"]).intersection(df.columns):
        df[c] = pd.to_numeric(df[c], errors="coerce")
    return df

# =============== CARGA MASIVA ===============
def cargar_uamex(base_dir: Path) -> pd.DataFrame:
    registros = []
    for reg_folder in os.listdir(base_dir):
        d = base_dir / reg_folder
        if not d.is_dir():
            continue
        for f in os.listdir(d):
            if not f.lower().endswith(".xlsx"):
                continue
            p = d / f
            region, years = parse_uamex_filename(str(p))
            if not region:
                continue
            df = robust_read_excel_uamex(p)
            df["Region"] = region
            df["Years"] = years
            registros.append(df)
    return pd.concat(registros, ignore_index=True) if registros else pd.DataFrame()

def cargar_unison(base_dir: Path) -> pd.DataFrame:
    registros = []
    for reg_folder in os.listdir(base_dir):
        d = base_dir / reg_folder
        if not d.is_dir():
            continue
        for f in os.listdir(d):
            if not f.lower().endswith(".csv"):
                continue
            p = d / f
            region, years = parse_unison_filename(str(p))
            if not region:
                continue
            df = robust_read_csv_unison(p)
            df["Region"] = region
            df["Years"] = years
            registros.append(df)
    return pd.concat(registros, ignore_index=True) if registros else pd.DataFrame()

# =============== CARGA ===============
df_uamex  = cargar_uamex(RUTA_SALIDA_UAMEX)
df_unison = cargar_unison(RUTA_SALIDA_UNISON)

print("UAMEX  :", df_uamex.shape,  sorted(df_uamex.Region.unique()) if not df_uamex.empty else [])
print("UNISON :", df_unison.shape, sorted(df_unison.Region.unique()) if not df_unison.empty else [])

# =============== GRÁFICAS (PDFs) ===============
sns.set_style("whitegrid")

def plot_region_period(region: str, years: str, save_png: bool = True) -> bool:
    """
    (No se usa en esta corrida) Dibuja líneas UAMEX vs UNISON para las variables disponibles comunes.
    Se mantiene por compatibilidad; no guardamos PNGs.
    """
    a = df_uamex[(df_uamex.Region == region) & (df_uamex.Years == years)].copy()
    b = df_unison[(df_unison.Region == region) & (df_unison.Years == years)].copy()
    if a.empty or b.empty:
        return False
    a_vars = [v for v in VARS if v in a.columns]
    b_vars = [v for v in VARS if v in b.columns]
    vars_use = sorted(set(a_vars).intersection(b_vars))
    if not vars_use:
        return False

    a_long = a[["Dia"] + vars_use].melt(id_vars="Dia", var_name="var", value_name="valor")
    a_long["Fuente"] = "UAMEX"
    b_long = b[["Dia"] + vars_use].melt(id_vars="Dia", var_name="var", value_name="valor")
    b_long["Fuente"] = "UNISON"
    df_plot = pd.concat([a_long, b_long], ignore_index=True)

    nvars = len(vars_use)
    ncols = 2
    nrows = int(np.ceil(nvars / ncols))
    fig, axes = plt.subplots(nrows, ncols, figsize=(14, 4 * nrows), sharex=True)
    axes = np.atleast_2d(axes).ravel()

    for i, v in enumerate(vars_use):
        ax = axes[i]
        sub = df_plot[df_plot["var"] == v]
        sns.lineplot(data=sub, x="Dia", y="valor", hue="Fuente", ax=ax)
        ax.set_title(f"{v} — {region} ({years})")
        ax.set_xlabel("Día del ciclo")
        ax.set_ylabel(v)
        ax.legend(loc="best")

    for j in range(i + 1, len(axes)):
        axes[j].set_visible(False)

    fig.tight_layout()
    # NO guardamos PNG
    plt.close(fig)
    return True

def exportar_pdf_por_region(region: str) -> int:
    """
    Genera un PDF por región con todas las temporadas en común entre UAMEX y UNISON.
    Devuelve el número de páginas (gráficas multipanel) escritas.
    """
    years_uamex  = set(df_uamex[df_uamex.Region == region]["Years"].unique())
    years_unison = set(df_unison[df_unison.Region == region]["Years"].unique())
    inter = sorted(years_uamex.intersection(years_unison))
    if not inter:
        return 0

    pdf_path = RUTA_COMP / f"{region.replace(' ', '_')}_comparacion.pdf"
    pages = 0
    with PdfPages(pdf_path) as pdf:
        for years in inter:
            a = df_uamex[(df_uamex.Region == region) & (df_uamex.Years == years)].copy()
            b = df_unison[(df_unison.Region == region) & (df_unison.Years == years)].copy()
            if a.empty or b.empty:
                continue

            vars_use = sorted(set([v for v in VARS if v in a.columns]).intersection([v for v in VARS if v in b.columns]))
            if not vars_use:
                continue

            a_long = a[["Dia"] + vars_use].melt(id_vars="Dia", var_name="var", value_name="valor")
            a_long["Fuente"] = "UAMEX"
            b_long = b[["Dia"] + vars_use].melt(id_vars="Dia", var_name="var", value_name="valor")
            b_long["Fuente"] = "UNISON"
            df_plot = pd.concat([a_long, b_long], ignore_index=True)

            nvars = len(vars_use)
            ncols = 2
            nrows = int(np.ceil(nvars / ncols))
            fig, axes = plt.subplots(nrows, ncols, figsize=(14, 4 * nrows), sharex=True)
            axes = np.atleast_2d(axes).ravel()
            for i, v in enumerate(vars_use):
                ax = axes[i]
                sub = df_plot[df_plot["var"] == v]
                sns.lineplot(data=sub, x="Dia", y="valor", hue="Fuente", ax=ax)
                ax.set_title(f"{v} — {region} ({years})")
                ax.set_xlabel("Día del ciclo")
                ax.set_ylabel(v)
                ax.legend(loc="best")
            for j in range(i + 1, len(axes)):
                axes[j].set_visible(False)
            fig.tight_layout()
            pdf.savefig(fig)
            plt.close(fig)
            pages += 1
    return pages

# =============== MÉTRICAS (MAE, RMSE, Bias) ===============
def resumen_diferencias(df_uamex: pd.DataFrame, df_unison: pd.DataFrame) -> pd.DataFrame:
    rows = []
    for region in sorted(set(df_uamex.Region.unique()).intersection(df_unison.Region.unique())):
        years_uamex  = set(df_uamex[df_uamex.Region == region]["Years"].unique())
        years_unison = set(df_unison[df_unison.Region == region]["Years"].unique())
        for years in sorted(years_uamex.intersection(years_unison)):
            a = df_uamex[(df_uamex.Region == region) & (df_uamex.Years == years)].copy()
            b = df_unison[(df_unison.Region == region) & (df_unison.Years == years)].copy()
            if a.empty or b.empty:
                continue
            vars_use = sorted(set(VARS).intersection(a.columns).intersection(b.columns))
            if not vars_use:
                continue
            m = pd.merge(a[["Dia"] + vars_use], b[["Dia"] + vars_use], on="Dia", suffixes=("_UAMEX", "_UNISON"))
            for v in vars_use:
                diff = (m[f"{v}_UNISON"] - m[f"{v}_UAMEX"]).astype(float)  # Bias = UNISON - UAMEX
                rows.append({
                    "Region": region,
                    "Years": years,
                    "Variable": v,
                    "MAE": diff.abs().mean(),
                    "RMSE": np.sqrt((diff**2).mean()),
                    "Bias": diff.mean(),
                    "N": diff.notna().sum(),
                })
    return pd.DataFrame(rows)

# =============== EJECUCIÓN: SOLO PDFs y DOS REPORTES (ET0 y Rnl) ===============

# 1) Preparación de trabajos
regiones = sorted(set(df_uamex.Region.unique()).intersection(df_unison.Region.unique()))
pares_region_years = []
for region in regiones:
    years_uamex  = set(df_uamex[df_uamex.Region == region]["Years"].unique())
    years_unison = set(df_unison[df_unison.Region == region]["Years"].unique())
    for years in sorted(years_uamex.intersection(years_unison)):
        pares_region_years.append((region, years))

# 2) Workers para procesos (solo PDFs)
def worker_pdf(region):
    try:
        pages = exportar_pdf_por_region(region)  # devuelve int (páginas)
        gc.collect()
        return int(pages) if pages is not None else 0
    except Exception:
        return 0

# 3) Ejecutores con procesos (contexto fork para Linux/notebooks)
ctx = mp.get_context("fork")
N_WORKERS = min(8, os.cpu_count() or 8)

# --- PDFs por región en paralelo ---
total_pages = 0
with ProcessPoolExecutor(max_workers=N_WORKERS, mp_context=ctx) as ex:
    futures = [ex.submit(worker_pdf, r) for r in regiones]
    for fut in as_completed(futures):
        total_pages += fut.result()
print(f"Páginas PDF generadas (todas las regiones): {total_pages}  → {RUTA_COMP}")

# --- Métricas (solo para ET0 y Rnl; NO se guarda el resumen completo) ---
df_metrics = resumen_diferencias(df_uamex, df_unison)

# Helper para exportar en el formato pedido
def exportar_variable(df, variable, nombre_salida):
    df_var = (
        df[df["Variable"] == variable]
          .loc[:, ["Region", "Years", "MAE", "RMSE", "Bias", "N"]]
          .rename(columns={"Years": "Ciclo", "N": "Dias del ciclo"})
          .sort_values(["Region", "Ciclo"], kind="mergesort")
          .reset_index(drop=True)
    )
    out_path = RUTA_COMP / nombre_salida
    df_var.to_csv(out_path, index=False)
    print(f"OK → Guardado: {out_path} ({len(df_var)} filas)")
    return df_var

# Generar SOLO los dos reportes pedidos
df_et0 = exportar_variable(df_metrics, "ET0", "Eto_uamex_vs_unison.csv")
df_rnl = exportar_variable(df_metrics, "Rnl", "Rnl_uamex_vs_unison.csv")


UAMEX  : (15685, 24) ['Cajeme', 'Ensenada', 'Etchojoa', 'Metepec', 'Toluca', 'Villa de Allende', 'Zapopan']
UNISON : (15685, 28) ['Cajeme', 'Ensenada', 'Etchojoa', 'Metepec', 'Toluca', 'Villa de Allende', 'Zapopan']
Páginas PDF generadas (todas las regiones): 102  → /lustre/home/mvalenzuela/Ocotillo/DataAqua/Comparaciones_FAO56_7
OK → Guardado: /lustre/home/mvalenzuela/Ocotillo/DataAqua/Comparaciones_FAO56_7/Eto_uamex_vs_unison.csv (102 filas)
OK → Guardado: /lustre/home/mvalenzuela/Ocotillo/DataAqua/Comparaciones_FAO56_7/Rnl_uamex_vs_unison.csv (102 filas)


In [2]:
# ========= PDF de métricas (ET0 y Rnl) con Bias-heatmap + líneas por ciclo =========
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from pathlib import Path

# --- Rutas (ya debes tener RUTA_COMP definida arriba) ---
try:
    RUTA_COMP
except NameError:
    raise RuntimeError("RUTA_COMP no está definida. Define RUTA_COMP = RUTA_BASE / 'Comparaciones_FAO56_4' arriba.")

# --- Cargar CSVs (ojo: 't' minúscula en Eto) ---
path_eto = RUTA_COMP / "Eto_uamex_vs_unison.csv"
path_rnl = RUTA_COMP / "Rnl_uamex_vs_unison.csv"
df_eto = pd.read_csv(path_eto)
df_rnl = pd.read_csv(path_rnl)

# --- Estilo ---
sns.set(style="whitegrid")

# --- Orden natural de 'Ciclo' (p.ej. 2010, 2010-2011, 2011, ...) ---
def ciclo_sort_key(y):
    y = str(y)
    parts = y.split("-")
    a = int(parts[0])
    b = int(parts[1]) if len(parts) > 1 else a
    return (a, b)

for df in (df_eto, df_rnl):
    df["Ciclo"] = df["Ciclo"].astype(str)

# --- Unidades por variable ---
UNITS = {
    "ET0": "mm d$^{-1}$",
    "Rnl": "MJ m$^{-2}$ d$^{-1}$",
}

def pdf_report(variable: str, df: pd.DataFrame, pdf_path: Path, cmap_heatmap: str):
    """
    Crea un PDF con 4 páginas:
      1) Heatmap de Bias por región x ciclo
      2) Bias por ciclo (líneas + puntos, color = región)
      3) MAE por ciclo (líneas + puntos, color = región)
      4) RMSE por ciclo (líneas + puntos, color = región)
    Guarda en pdf_path. No usa plt.show().
    """
    units = UNITS.get(variable, "")
    with PdfPages(pdf_path) as pdf:

        # --- Página 1: Heatmap de Bias (región x ciclo) ---
        pivot_bias = (df
                      .pivot(index="Region", columns="Ciclo", values="Bias")
                      .reindex(index=sorted(df["Region"].unique()),
                               columns=sorted(df["Ciclo"].unique(), key=ciclo_sort_key)))
        fig1, ax1 = plt.subplots(figsize=(max(12, 0.6*pivot_bias.shape[1]),
                                          max(5, 0.6*pivot_bias.shape[0])))
        # mapa centrado en 0 para ver positivos/negativos
        vlim = max(abs(pivot_bias.min().min()), abs(pivot_bias.max().max()))
        sns.heatmap(pivot_bias, annot=True, cmap="coolwarm", center=0, vmin=-vlim, vmax=vlim,
                    fmt=".2f", linewidths=0.5, linecolor="gray",
                    cbar_kws={"label": f"Bias (UNISON − UAMEX) [{units}]"}, ax=ax1)
        ax1.set_title(f"Bias de {variable} por Región y Ciclo")
        ax1.set_xlabel("Ciclo"); ax1.set_ylabel("Región")
        fig1.tight_layout()
        pdf.savefig(fig1); plt.close(fig1)

        # --- Orden temporal para líneas ---
        df_sorted = df.sort_values("Ciclo", key=lambda s: s.map(ciclo_sort_key))

        # --- Página 2: Bias por ciclo (líneas + puntos) ---
        fig2, ax2 = plt.subplots(figsize=(12, 6))
        sns.lineplot(data=df_sorted, x="Ciclo", y="Bias", hue="Region", marker="o", ax=ax2)
        ax2.axhline(0, color="black", linestyle="--", linewidth=1)
        ax2.set_title(f"Bias de {variable} por Región y Ciclo")
        ax2.set_ylabel(f"Bias (UNISON − UAMEX) [{units}]"); ax2.set_xlabel("Ciclo")
        plt.setp(ax2.get_xticklabels(), rotation=45, ha="right")
        fig2.tight_layout()
        pdf.savefig(fig2); plt.close(fig2)

        # --- Página 3: MAE por ciclo (líneas + puntos) ---
        fig3, ax3 = plt.subplots(figsize=(12, 6))
        sns.lineplot(data=df_sorted, x="Ciclo", y="MAE", hue="Region", marker="o", ax=ax3)
        ax3.set_title(f"MAE de {variable} por Región y Ciclo")
        ax3.set_ylabel(f"MAE [{units}]"); ax3.set_xlabel("Ciclo")
        plt.setp(ax3.get_xticklabels(), rotation=45, ha="right")
        fig3.tight_layout()
        pdf.savefig(fig3); plt.close(fig3)

        # --- Página 4: RMSE por ciclo (líneas + puntos) ---
        fig4, ax4 = plt.subplots(figsize=(12, 6))
        sns.lineplot(data=df_sorted, x="Ciclo", y="RMSE", hue="Region", marker="o", ax=ax4)
        ax4.set_title(f"RMSE de {variable} por Región y Ciclo")
        ax4.set_ylabel(f"RMSE [{units}]"); ax4.set_xlabel("Ciclo")
        plt.setp(ax4.get_xticklabels(), rotation=45, ha="right")
        fig4.tight_layout()
        pdf.savefig(fig4); plt.close(fig4)

    print(f"[OK] PDF guardado: {pdf_path}")

# --- Generar PDFs ---
pdf_eto = RUTA_COMP / "report_ET0_metricas.pdf"
pdf_rnl = RUTA_COMP / "report_Rnl_metricas.pdf"

pdf_report("ET0", df_eto, pdf_eto, cmap_heatmap="YlOrRd")
pdf_report("Rnl", df_rnl, pdf_rnl, cmap_heatmap="YlGnBu")

[OK] PDF guardado: /lustre/home/mvalenzuela/Ocotillo/DataAqua/Comparaciones_FAO56_7/report_ET0_metricas.pdf
[OK] PDF guardado: /lustre/home/mvalenzuela/Ocotillo/DataAqua/Comparaciones_FAO56_7/report_Rnl_metricas.pdf


In [3]:
# # ========= PDF de métricas (ET0 y Rnl) a partir de los CSV generados =========
# import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt
# from matplotlib.backends.backend_pdf import PdfPages
# from pathlib import Path

# # --- Asegurar rutas --- (usa tu RUTA_COMP ya definida más arriba)
# try:
#     RUTA_COMP
# except NameError:
#     raise RuntimeError("RUTA_COMP no está definida. Asegúrate de haberla fijado a 'Comparaciones_FAO56_4' antes.")

# # --- Cargar CSVs (ojo: 't' minúscula en Eto) ---
# path_eto = RUTA_COMP / "Eto_uamex_vs_unison.csv"
# path_rnl = RUTA_COMP / "Rnl_uamex_vs_unison.csv"
# df_eto = pd.read_csv(path_eto)
# df_rnl = pd.read_csv(path_rnl)

# # --- Estilo ---
# sns.set(style="whitegrid")

# # --- Orden natural de 'Ciclo' (p.ej. 2010, 2010-2011, 2011, ...) ---
# def ciclo_sort_key(y):
#     y = str(y)
#     parts = y.split("-")
#     a = int(parts[0])
#     b = int(parts[1]) if len(parts) > 1 else a
#     return (a, b)

# for df in (df_eto, df_rnl):
#     df["Ciclo"] = df["Ciclo"].astype(str)

# def pdf_report(variable: str, df: pd.DataFrame, pdf_path: Path, cmap_heatmap: str):
#     """
#     Crea un PDF con 3 páginas:
#       1) Barras de Bias por región (promedio)
#       2) Heatmap de MAE por región x ciclo
#       3) RMSE por ciclo (líneas) con color por región
#     Guarda en pdf_path. No usa plt.show().
#     """
#     with PdfPages(pdf_path) as pdf:

#         # Página 1: Bias por región
#         fig1, ax1 = plt.subplots(figsize=(12, 6))
#         sns.barplot(data=df, x="Region", y="Bias", errorbar=None, ax=ax1)
#         ax1.axhline(0, color="black", linestyle="--", linewidth=1)
#         ax1.set_title(f"Sesgo (Bias) de {variable} — UAMEX vs UNISON")
#         ax1.set_ylabel("Bias (UNISON − UAMEX)")
#         ax1.set_xlabel("Región")
#         plt.setp(ax1.get_xticklabels(), rotation=45, ha="right")
#         fig1.tight_layout()
#         pdf.savefig(fig1); plt.close(fig1)

#         # Página 2: Heatmap MAE por región x ciclo
#         pivot_mae = (df
#                      .pivot(index="Region", columns="Ciclo", values="MAE")
#                      .reindex(index=sorted(df["Region"].unique()),
#                               columns=sorted(df["Ciclo"].unique(), key=ciclo_sort_key)))
#         fig2, ax2 = plt.subplots(figsize=(max(12, 0.6*pivot_mae.shape[1]),
#                                           max(5, 0.6*pivot_mae.shape[0])))
#         sns.heatmap(pivot_mae, annot=True, cmap=cmap_heatmap, fmt=".2f",
#                     linewidths=0.5, linecolor="gray", cbar_kws={"label": "MAE"}, ax=ax2)
#         ax2.set_title(f"MAE de {variable} por Región y Ciclo")
#         ax2.set_xlabel("Ciclo"); ax2.set_ylabel("Región")
#         fig2.tight_layout()
#         pdf.savefig(fig2); plt.close(fig2)

#         # Página 3: RMSE por ciclo y región (líneas)
#         df_sorted = df.sort_values("Ciclo", key=lambda s: s.map(ciclo_sort_key))
#         fig3, ax3 = plt.subplots(figsize=(12, 6))
#         sns.lineplot(data=df_sorted, x="Ciclo", y="RMSE", hue="Region", marker="o", ax=ax3)
#         ax3.set_title(f"RMSE de {variable} por Región y Ciclo")
#         ax3.set_ylabel("RMSE"); ax3.set_xlabel("Ciclo")
#         plt.setp(ax3.get_xticklabels(), rotation=45, ha="right")
#         fig3.tight_layout()
#         pdf.savefig(fig3); plt.close(fig3)

#     print(f"[OK] PDF guardado: {pdf_path}")

# # --- Generar PDFs ---
# pdf_eto = RUTA_COMP / "report_ET0_metricas.pdf"
# pdf_rnl = RUTA_COMP / "report_Rnl_metricas.pdf"

# pdf_report("ET0", df_eto, pdf_eto, cmap_heatmap="YlOrRd")
# pdf_report("Rnl", df_rnl, pdf_rnl, cmap_heatmap="YlGnBu")

## ¿Qué significan y en qué unidades están?

- **ET0**: milímetros por día → `mm d⁻¹`  
- **Rnl** (radiación neta de onda larga, diaria): megajoules por metro cuadrado por día → `MJ m⁻² d⁻¹`  

Como **MAE**, **RMSE** y **Bias** se calculan sobre la misma variable, sus unidades son las mismas:

- Si comparas **ET0** → MAE/RMSE/Bias en `mm d⁻¹`  
- Si comparas **Rnl** → MAE/RMSE/Bias en `MJ m⁻² d⁻¹`  

---

## ¿Cuándo son “buenos” o “malos”?

### ET0 (`mm d⁻¹`)
- **Muy bueno**: MAE o RMSE ≤ 0.3  
- **Aceptable**: 0.3 – 0.5  
- **Regular**: 0.5 – 1.0  
- **Malo**: > 1.0  

### Rnl (`MJ m⁻² d⁻¹`)  
(la Rnl diaria típica está ≈ 0–5 MJ m⁻² d⁻¹ en muchos climas)  
- **Muy bueno**: MAE o RMSE ≤ 0.3  
- **Aceptable**: 0.3 – 0.7  
- **Regular**: 0.7 – 1.5  
- **Malo**: > 1.5  

---

## Bias (UNISON − UAMEX)
- **Positivo** → UNISON da valores más altos (sobreestima vs UAMEX)  
- **Negativo** → UNISON da valores más bajos (subestima vs UAMEX)  
- **Cerca de 0** → sin sesgo sistemático  

---

⚠️ En tus resultados, los Bias/MAE de **Rnl** ~2–3 `MJ m⁻² d⁻¹` son **grandes**: indican una **diferencia metodológica fuerte** (FAO-56 Eq.39 vs usar LW de NASA), no un “ruido” pequeño.


## Métricas de evaluación

### 🔹 Bias (sesgo promedio)
- Es el **promedio de las diferencias** entre las dos fuentes (UNISON – UAMEX en tu código actual).  
- **Interpretación**:
  - **Bias positivo** → en promedio UNISON da valores más grandes que UAMEX.  
  - **Bias negativo** → en promedio UAMEX da valores más grandes que UNISON.  
  - **Bias cercano a 0** → no hay tendencia clara, ambos métodos son consistentes.  
  - **Bias grande** (positivo o negativo) → hay una tendencia sistemática de una fuente a **sobreestimar** o **subestimar**.  

---

### 🔹 MAE (Mean Absolute Error, error absoluto medio)
- Es el **promedio de las diferencias absolutas** `|UNISON – UAMEX|`.  
- Siempre **≥ 0**.  
- Fácil de interpretar:  
  - Ejemplo: si **MAE = 0.5 mm/día**, significa que en promedio se equivocan en **medio milímetro por día**.  
- **Mientras más pequeño → mejor.**

---

### 🔹 RMSE (Root Mean Square Error, raíz del error cuadrático medio)
- Similar al MAE, pero **penaliza más fuerte los errores grandes** (porque eleva al cuadrado antes de promediar).  
- Siempre **≥ 0**.  
- **Interpretación**:
  - Si **RMSE ≈ MAE** → los errores son consistentes.  
  - Si **RMSE ≫ MAE** → hay algunos **outliers** o errores muy grandes que elevan el promedio cuadrático.  
- **Mientras más pequeño → mejor.**

---

### 🔹 ¿Qué valores son “buenos” o “malos”?
- No hay regla absoluta, depende de la **variable** y de la **aplicación**.  
- **ET0** (`mm/día`):
  - Bias ≈ 0 → deseable.  
  - **MAE y RMSE ≤ 0.5 mm/día** → suelen considerarse aceptables en validaciones climatológicas.  
- **Rnl** (`W/m²` o `MJ m⁻² d⁻¹` dependiendo la forma reportada):
  - Valores típicos: ~100–400 W/m² (≈ 0–5 MJ m⁻² d⁻¹ en promedio diario).  
  - **MAE de 5 W/m²** (≈ 0.1 MJ m⁻² d⁻¹) → muy bueno.  
  - **MAE de 30 W/m²** (≈ 0.6 MJ m⁻² d⁻¹) → ya empieza a preocupar.  

---

✅ En general: **más cerca de 0 = mejor**.
